# Homework 4: Data Transformation with dplyr - Part 2 (Mutate, Summarize, Group_by, Count)

**Student Name:** Solution Key

**Due Date:** [Date]

This is the complete solution for Homework 4.

## Part 1: Setup and Data Import

In [3]:
setwd("/Users/humphrjk/GitHub/ai-homework-grader-clean/data")
# Load required packages
library(tidyverse)

# Import the company sales data
company_data <- read_csv("company_sales_data.csv")

# Examine the dataset structure
str(company_data)

# Display the first few rows
head(company_data, 6)

# Generate summary statistics
summary(company_data)

# Check dataset dimensions
cat("Dataset dimensions:", nrow(company_data), "rows x", ncol(company_data), "columns\n")

Rows: 300 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): Sales_Rep_Name, Region, Product_Category
dbl  (4): TransactionID, Revenue, Cost, Units_Sold
date (1): Sale_Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


spc_tbl_ [300 × 8] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ TransactionID   : num [1:300] 1 2 3 4 5 6 7 8 9 10 ...
 $ Sales_Rep_Name  : chr [1:300] "Carol Davis" "Carol Davis" "Carol Davis" "Bob Smith" ...
 $ Region          : chr [1:300] "Latin America" "Europe" "Europe" "Europe" ...
 $ Product_Category: chr [1:300] "Services" "Hardware" "Services" "Hardware" ...
 $ Revenue         : num [1:300] 20751 32360 39268 28865 3932 ...
 $ Cost            : num [1:300] 12253 24595 23291 12429 1778 ...
 $ Units_Sold      : num [1:300] 78 13 34 90 63 26 25 1 20 15 ...
 $ Sale_Date       : Date[1:300], format: "2023-04-24" "2023-06-09" ...
 - attr(*, "spec")=
  .. cols(
  ..   TransactionID = col_double(),
  ..   Sales_Rep_Name = col_character(),
  ..   Region = col_character(),
  ..   Product_Category = col_character(),
  ..   Revenue = col_double(),
  ..   Cost = col_double(),
  ..   Units_Sold = col_double(),
  ..   Sale_Date = col_date(format = "")
  .. )
 - attr(*, "problems")=<externalptr

TransactionID,Sales_Rep_Name,Region,Product_Category,Revenue,Cost,Units_Sold,Sale_Date
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<date>
1,Carol Davis,Latin America,Services,20750.92,12253.36,78,2023-04-24
2,Carol Davis,Europe,Hardware,32359.98,24595.20,13,2023-06-09
3,Carol Davis,Europe,Services,39268.40,23291.09,34,2023-03-25
4,Bob Smith,Europe,Hardware,28865.09,12428.74,90,2023-04-11
5,Frank Miller,Latin America,Software,3932.36,1778.18,63,2023-08-26
6,Carol Davis,Latin America,Services,48209.75,26052.04,26,2023-09-04


 TransactionID    Sales_Rep_Name        Region          Product_Category  
 Min.   :  1.00   Length:300         Length:300         Length:300        
 1st Qu.: 75.75   Class :character   Class :character   Class :character  
 Median :150.50   Mode  :character   Mode  :character   Mode  :character  
 Mean   :150.50                                                           
 3rd Qu.:225.25                                                           
 Max.   :300.00                                                           
    Revenue           Cost         Units_Sold       Sale_Date         
 Min.   : 1032   Min.   :  567   Min.   :  1.00   Min.   :2023-01-02  
 1st Qu.:15034   1st Qu.: 7216   1st Qu.: 28.75   1st Qu.:2023-04-07  
 Median :26062   Median :13956   Median : 56.00   Median :2023-07-05  
 Mean   :25906   Mean   :14547   Mean   : 53.90   Mean   :2023-07-07  
 3rd Qu.:37708   3rd Qu.:21714   3rd Qu.: 80.00   3rd Qu.:2023-10-08  
 Max.   :49956   Max.   :37811   Max.   :100.00  

Dataset dimensions: 300 rows x 8 columns


## Part 2: Creating New Variables with mutate()

In [4]:
# Create basic financial metrics using mutate()
company_data <- company_data %>%
  mutate(
    # Create Profit (Revenue - Cost)
    Profit = Revenue - Cost,
    
    # Create Profit_Margin ((Profit / Revenue) * 100)
    Profit_Margin = (Profit / Revenue) * 100,
    
    # Create Cost_Ratio ((Cost / Revenue) * 100)
    Cost_Ratio = (Cost / Revenue) * 100,
    
    # Create Revenue_Per_Unit (Revenue / Units_Sold)
    Revenue_Per_Unit = Revenue / Units_Sold,
    
    # Create Cost_Per_Unit (Cost / Units_Sold)
    Cost_Per_Unit = Cost / Units_Sold,
    
    # Create ROI ((Profit / Cost) * 100)
    ROI = (Profit / Cost) * 100
  )

# Display the new financial metrics
company_data %>%
  select(Revenue, Cost, Profit, Profit_Margin, ROI) %>%
  head(10)

Revenue,Cost,Profit,Profit_Margin,ROI
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
20750.92,12253.36,8497.56,40.95028,69.34882
32359.98,24595.20,7764.78,23.99501,31.57031
39268.40,23291.09,15977.31,40.68745,68.59838
28865.09,12428.74,16436.35,56.94197,132.24470
3932.36,1778.18,2154.18,54.78084,121.14522
48209.75,26052.04,22157.71,45.96106,85.05173
33055.35,20432.24,12623.11,38.18780,61.78035
49856.54,37118.63,12737.91,25.54913,34.31676
13477.13,8685.85,4791.28,35.55119,55.16190


## Part 3: Creating Categorical Variables

In [5]:
# Create categorical variables for business segmentation
company_data <- company_data %>%
  mutate(
    # Create Performance_Category using case_when()
    Performance_Category = case_when(
      Profit_Margin > 50 ~ "High",
      Profit_Margin > 30 ~ "Medium",
      TRUE ~ "Low"
    ),
    
    # Create Revenue_Size category
    Revenue_Size = case_when(
      Revenue > 30000 ~ "Large",
      Revenue > 15000 ~ "Medium",
      TRUE ~ "Small"
    ),
    
    # Create Deal_Size category
    Deal_Size = case_when(
      Units_Sold > 50 ~ "Bulk",
      Units_Sold > 20 ~ "Standard",
      TRUE ~ "Small"
    ),
    
    # Create High_Value_Customer flag
    High_Value_Customer = ifelse(Revenue > 25000, "Yes", "No"),
    
    # Create Profitable_Deal flag
    Profitable_Deal = ifelse(Profit_Margin > 40, "Yes", "No")
  )

# Examine the distribution of categorical variables
cat("Performance Category Distribution:\n")
table(company_data$Performance_Category)

cat("\nRevenue Size Distribution:\n")
table(company_data$Revenue_Size)

cat("\nHigh Value Customer Distribution:\n")
table(company_data$High_Value_Customer)

Performance Category Distribution:



  High    Low Medium 
   113     66    121 


Revenue Size Distribution:



 Large Medium  Small 
   122    104     74 


High Value Customer Distribution:



 No Yes 
146 154 

## Part 4: Summary Statistics with summarize()

In [6]:
# Calculate overall business summary statistics
business_summary <- company_data %>%
  summarize(
    # Calculate total_revenue
    total_revenue = sum(Revenue, na.rm = TRUE),
    
    # Calculate total_profit
    total_profit = sum(Profit, na.rm = TRUE),
    
    # Calculate avg_profit_margin
    avg_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    
    # Calculate avg_roi
    avg_roi = mean(ROI, na.rm = TRUE),
    
    # Calculate total_units
    total_units = sum(Units_Sold, na.rm = TRUE),
    
    # Calculate transaction_count
    transaction_count = n(),
    
    # Calculate avg_revenue_per_transaction
    avg_revenue_per_transaction = mean(Revenue, na.rm = TRUE)
  )

# Display the business summary
print("Overall Business Performance Summary:")
print(business_summary)

# Calculate detailed revenue statistics
revenue_statistics <- company_data %>%
  summarize(
    min_revenue = min(Revenue, na.rm = TRUE),
    max_revenue = max(Revenue, na.rm = TRUE),
    mean_revenue = mean(Revenue, na.rm = TRUE),
    median_revenue = median(Revenue, na.rm = TRUE),
    sd_revenue = sd(Revenue, na.rm = TRUE)
  )

print("\nDetailed Revenue Statistics:")
print(revenue_statistics)

[1] "Overall Business Performance Summary:"
# A tibble: 1 × 7
  total_revenue total_profit avg_profit_margin avg_roi total_units
          <dbl>        <dbl>             <dbl>   <dbl>       <dbl>
1      7771711.     3407512.              44.2    93.5       16169
# ℹ 2 more variables: transaction_count <int>,
#   avg_revenue_per_transaction <dbl>
[1] "\nDetailed Revenue Statistics:"
# A tibble: 1 × 5
  min_revenue max_revenue mean_revenue median_revenue sd_revenue
        <dbl>       <dbl>        <dbl>          <dbl>      <dbl>
1       1032.      49956.       25906.         26062.     13944.


## Part 5: Grouped Analysis with group_by()

In [7]:
# Regional Analysis
regional_performance <- company_data %>%
  group_by(Region) %>%
  summarize(
    total_revenue = sum(Revenue, na.rm = TRUE),
    total_profit = sum(Profit, na.rm = TRUE),
    avg_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    transaction_count = n(),
    total_units = sum(Units_Sold, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  mutate(
    revenue_share = (total_revenue / sum(total_revenue)) * 100
  ) %>%
  arrange(desc(total_revenue))

print("Regional Performance Analysis:")
print(regional_performance)

[1] "Regional Performance Analysis:"
# A tibble: 4 × 7
  Region        total_revenue total_profit avg_profit_margin transaction_count
  <chr>                 <dbl>        <dbl>             <dbl>             <int>
1 Europe             2224182.     1006807.              45.5                82
2 Latin America      2112037.      891481.              43.0                83
3 Asia Pacific       1804243.      827243.              46.0                67
4 North America      1631248.      681981.              42.4                68
# ℹ 2 more variables: total_units <dbl>, revenue_share <dbl>


In [8]:
# Product Category Analysis
category_performance <- company_data %>%
  group_by(Product_Category) %>%
  summarize(
    total_revenue = sum(Revenue, na.rm = TRUE),
    total_profit = sum(Profit, na.rm = TRUE),
    avg_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    transaction_count = n(),
    total_units = sum(Units_Sold, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  mutate(
    revenue_share = (total_revenue / sum(total_revenue)) * 100
  ) %>%
  arrange(desc(total_revenue))

print("Product Category Performance Analysis:")
print(category_performance)

[1] "Product Category Performance Analysis:"
# A tibble: 4 × 7
  Product_Category total_revenue total_profit avg_profit_margin
  <chr>                    <dbl>        <dbl>             <dbl>
1 Consulting            1978840.      884265.              44.4
2 Services              1961565.      814908.              42.3
3 Hardware              1951325.      858247.              43.5
4 Software              1879981.      850092.              46.4
# ℹ 3 more variables: transaction_count <int>, total_units <dbl>,
#   revenue_share <dbl>


In [9]:
# Performance Category Analysis
performance_analysis <- company_data %>%
  group_by(Performance_Category) %>%
  summarize(
    total_revenue = sum(Revenue, na.rm = TRUE),
    total_profit = sum(Profit, na.rm = TRUE),
    avg_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    transaction_count = n(),
    total_units = sum(Units_Sold, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  mutate(
    revenue_share = (total_revenue / sum(total_revenue)) * 100
  )

print("Performance Category Analysis:")
print(performance_analysis)

[1] "Performance Category Analysis:"
# A tibble: 3 × 7
  Performance_Category total_revenue total_profit avg_profit_margin
  <chr>                        <dbl>        <dbl>             <dbl>
1 High                      2771248.     1682947.              60.1
2 Low                       1813077.      457363.              25.3
3 Medium                    3187386.     1267202.              39.8
# ℹ 3 more variables: transaction_count <int>, total_units <dbl>,
#   revenue_share <dbl>


## Part 6: Advanced Grouping and Cross-Tabulation

In [10]:
# Multi-dimensional analysis: Region and Product Category
region_category_analysis <- company_data %>%
  group_by(Region, Product_Category) %>%
  summarize(
    total_revenue = sum(Revenue, na.rm = TRUE),
    transaction_count = n(),
    avg_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  arrange(desc(total_revenue))

print("Top 10 Region-Product Category Combinations:")
print(head(region_category_analysis, 10))

[1] "Top 10 Region-Product Category Combinations:"
# A tibble: 10 × 5
   Region     Product_Category total_revenue transaction_count avg_profit_margin
   <chr>      <chr>                    <dbl>             <int>             <dbl>
 1 Europe     Hardware               777044.                27              45.6
 2 Asia Paci… Consulting             759641.                27              47.5
 3 Latin Ame… Services               644772.                22              43.9
 4 Latin Ame… Software               559611.                24              44.2
 5 Europe     Software               542961.                23              49.6
 6 Europe     Services               513507.                18              42.8
 7 Latin Ame… Hardware               474257.                19              43.1
 8 North Ame… Services               471460.                17              39.5
 9 Asia Paci… Software               440797.                15              47.6
10 Latin Ame… Consulting             43

In [11]:
# Frequency analysis using count()
print("Frequency Analysis:")

# Count by Performance_Category
performance_counts <- company_data %>%
  count(Performance_Category, sort = TRUE)

print("Performance Category Distribution:")
print(performance_counts)

# Count by Revenue_Size
revenue_size_counts <- company_data %>%
  count(Revenue_Size, sort = TRUE)

print("\nRevenue Size Distribution:")
print(revenue_size_counts)

# Count by Deal_Size
deal_size_counts <- company_data %>%
  count(Deal_Size, sort = TRUE)

print("\nDeal Size Distribution:")
print(deal_size_counts)

[1] "Frequency Analysis:"
[1] "Performance Category Distribution:"
# A tibble: 3 × 2
  Performance_Category     n
  <chr>                <int>
1 Medium                 121
2 High                   113
3 Low                     66
[1] "\nRevenue Size Distribution:"
# A tibble: 3 × 2
  Revenue_Size     n
  <chr>        <int>
1 Large          122
2 Medium         104
3 Small           74
[1] "\nDeal Size Distribution:"
# A tibble: 3 × 2
  Deal_Size     n
  <chr>     <int>
1 Bulk        159
2 Standard     90
3 Small        51


In [12]:
# Cross-tabulation analysis
print("Cross-Tabulation Analysis:")

# Create a cross-tabulation of Performance_Category vs Revenue_Size
cross_tab_performance_revenue <- company_data %>%
  count(Performance_Category, Revenue_Size)

print("Performance Category vs Revenue Size:")
print(cross_tab_performance_revenue)

# Alternative using base R table function
print("\nUsing table() function:")
table(company_data$Performance_Category, company_data$Revenue_Size)

[1] "Cross-Tabulation Analysis:"
[1] "Performance Category vs Revenue Size:"
# A tibble: 9 × 3
  Performance_Category Revenue_Size     n
  <chr>                <chr>        <int>
1 High                 Large           41
2 High                 Medium          38
3 High                 Small           34
4 Low                  Large           28
5 Low                  Medium          30
6 Low                  Small            8
7 Medium               Large           53
8 Medium               Medium          36
9 Medium               Small           32
[1] "\nUsing table() function:"


        
         Large Medium Small
  High      41     38    34
  Low       28     30     8
  Medium    53     36    32

## Part 7: Business Intelligence Dashboard

In [13]:
# Create a comprehensive business intelligence dashboard
cat("\n", rep("=", 60), "\n")
cat("              BUSINESS INTELLIGENCE DASHBOARD\n")
cat(rep("=", 60), "\n\n")

# Section 1: Key Performance Indicators
cat("📊 KEY PERFORMANCE INDICATORS\n")
cat("─────────────────────────────────\n")

kpi_summary <- company_data %>%
  summarize(
    total_revenue = sum(Revenue, na.rm = TRUE),
    total_profit = sum(Profit, na.rm = TRUE),
    overall_profit_margin = mean(Profit_Margin, na.rm = TRUE),
    total_transactions = n(),
    avg_deal_size = mean(Revenue, na.rm = TRUE),
    high_value_rate = sum(High_Value_Customer == "Yes") / n() * 100,
    profitable_deal_rate = sum(Profitable_Deal == "Yes") / n() * 100
  )

cat("Total Revenue: $", format(kpi_summary$total_revenue, big.mark = ","), "\n")
cat("Total Profit: $", format(kpi_summary$total_profit, big.mark = ","), "\n")
cat("Overall Profit Margin:", round(kpi_summary$overall_profit_margin, 2), "%\n")
cat("Total Transactions:", kpi_summary$total_transactions, "\n")
cat("Average Deal Size: $", format(kpi_summary$avg_deal_size, big.mark = ","), "\n")
cat("High-Value Customer Rate:", round(kpi_summary$high_value_rate, 2), "%\n")
cat("Profitable Deal Rate:", round(kpi_summary$profitable_deal_rate, 2), "%\n")

# Section 2: Top Performers
cat("\n🏆 TOP PERFORMERS\n")
cat("─────────────────\n")

top_region <- regional_performance %>% slice(1)
cat("Top Region:", top_region$Region, "\n")
cat("  Revenue: $", format(top_region$total_revenue, big.mark = ","), "\n")

top_category <- category_performance %>% slice(1)
cat("\nTop Product Category:", top_category$Product_Category, "\n")
cat("  Revenue: $", format(top_category$total_revenue, big.mark = ","), "\n")

# Performance distribution
cat("\n📈 PERFORMANCE DISTRIBUTION\n")
cat("──────────────────────────\n")
perf_dist <- company_data %>%
  count(Performance_Category) %>%
  mutate(percentage = n / sum(n) * 100)

for(i in 1:nrow(perf_dist)) {
  cat(perf_dist$Performance_Category[i], ": ", perf_dist$n[i], 
      " (", round(perf_dist$percentage[i], 1), "%)\n", sep="")
}


 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
              BUSINESS INTELLIGENCE DASHBOARD
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

📊 KEY PERFORMANCE INDICATORS
─────────────────────────────────
Total Revenue: $ 7,771,711 
Total Profit: $ 3,407,512 
Overall Profit Margin: 44.22 %
Total Transactions: 300 
Average Deal Size: $ 25,905.7 
High-Value Customer Rate: 51.33 %
Profitable Deal Rate: 56 %

🏆 TOP PERFORMERS
─────────────────
Top Region: Europe 
  Revenue: $ 2,224,182 

Top Product Category: Consulting 
  Revenue: $ 1,978,840 

📈 PERFORMANCE DISTRIBUTION
──────────────────────────
High: 113 (37.7%)
Low: 66 (22%)
Medium: 121 (40.3%)


## Part 8: Data Validation and Quality Checks

In [14]:
# Comprehensive Data Validation
cat("📋 DATA VALIDATION REPORT\n")
cat(rep("=", 40), "\n\n")

# SECTION 1: Calculation Verification
cat("🧮 CALCULATION VERIFICATION\n")
cat(rep("─", 40), "\n")

# Validate profit calculation
profit_validation <- all.equal(company_data$Profit, company_data$Revenue - company_data$Cost)
cat("✓ Profit calculation: ")
cat(ifelse(profit_validation == TRUE, "✅ PASSED", "❌ FAILED"), "\n")

# Validate profit margin calculation
expected_profit_margin <- (company_data$Profit / company_data$Revenue) * 100
margin_validation <- all.equal(company_data$Profit_Margin, expected_profit_margin)
cat("✓ Profit Margin calculation: ")
cat(ifelse(margin_validation == TRUE, "✅ PASSED", "❌ FAILED"), "\n")

# Validate ROI calculation
expected_roi <- (company_data$Profit / company_data$Cost) * 100
roi_validation <- all.equal(company_data$ROI, expected_roi)
cat("✓ ROI calculation: ")
cat(ifelse(roi_validation == TRUE, "✅ PASSED", "❌ FAILED"), "\n")

# SECTION 2: Range and Outlier Analysis
cat("\n📊 RANGE AND OUTLIER VALIDATION\n")
cat(rep("─", 40), "\n")

extreme_margins <- sum(company_data$Profit_Margin > 100 | company_data$Profit_Margin < -50, na.rm = TRUE)
cat("⚠️  Extreme profit margins: ", extreme_margins, " transactions\n")

negative_revenue <- sum(company_data$Revenue < 0, na.rm = TRUE)
cat("⚠️  Negative revenue values: ", negative_revenue, " transactions\n")

invalid_units <- sum(company_data$Units_Sold <= 0, na.rm = TRUE)
cat("⚠️  Invalid units sold: ", invalid_units, " transactions\n")

# SECTION 3: Missing Data Analysis
cat("\n🔍 MISSING DATA ANALYSIS\n")
cat(rep("─", 40), "\n")

cat("Missing value analysis:\n")
cat("  • Missing Revenue: ", sum(is.na(company_data$Revenue)), "\n")
cat("  • Missing Cost: ", sum(is.na(company_data$Cost)), "\n")
cat("  • Missing Units_Sold: ", sum(is.na(company_data$Units_Sold)), "\n")
cat("  • Missing Profit: ", sum(is.na(company_data$Profit)), "\n")

# SECTION 4: Summary Statistics
cat("\n📈 SUMMARY STATISTICS\n")
cat(rep("─", 40), "\n")

cat("\nProfit Margin Distribution:\n")
print(summary(company_data$Profit_Margin))

cat("\nROI Distribution:\n")
print(summary(company_data$ROI))

cat("\n✅ Data validation complete!\n")

📋 DATA VALIDATION REPORT
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

🧮 CALCULATION VERIFICATION
─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
✓ Profit calculation: ✅ PASSED 
✓ Profit Margin calculation: ✅ PASSED 
✓ ROI calculation: ✅ PASSED 

📊 RANGE AND OUTLIER VALIDATION
─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
⚠️  Extreme profit margins:  0  transactions
⚠️  Negative revenue values:  0  transactions
⚠️  Invalid units sold:  0  transactions

🔍 MISSING DATA ANALYSIS
─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
Missing value analysis:
  • Missing Revenue:  0 
  • Missing Cost:  0 
  • Missing Units_Sold:  0 
  • Missing Profit:  0 

📈 SUMMARY STATISTICS
─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 

Profit Margin Distribution:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  20.13   31.17   43.70   44.2